<div class='bar_title'></div>

*Decision Support Systems*

# PuLP Introduction

Toni Greif<br>
Chair of Information Systems and Management

Winter Semester 19/20

## Using Drive to save Colab notebooks

The default location for storing your data is Google Drive. When you choose File → Save, the content you create goes to the root directory of your Google Drive. If you want to save the content to a different folder, you need to select that folder in Google Drive.

In WueCampus you find a introduction for saving and opening notebooks via Google Drive 

Before each PuLP implementation the package must be installed (not necessary if you don't use the Colab enviroment) and imported.

In [2]:
!pip install pulp
from pulp import *
import pandas as pd

## Example

A chocolate manufacturing company produces two types of chocolate – A and B. Both the chocolates require Milk and Choco only,


- each unit of Milk costs \\$0.1 and each unit of Choco \\$0.3
- each unit of A requires 1 unit of Milk and 2 units of Choco,
- each unit of B requires 3 unit of Milk and 2 units of Choco.

The company has a total of 20 units of Milk and 40 units of Choco.

The company sells
- each unit of A for \\$9
- each unit of B for \\$14

How many units of A and B should the company produce  to maximize its profit?

Our typical workflow will be the following:


- Read the text carefully and try to identify the following elements
    1. parameters
    2. decision variables
    3. objective
    4. constraints
    
    
- Try to formulate the complete optimization problem by hand

- Implement your model with the following steps
    1. create the model
    2. set the parameters
    3. create the decision variables
    4. set the objective
    5. set the constraints
    6. solve the problem
    7. print the solution

So let's start with our first model...

In [3]:
# Create model
prob = LpProblem("chocolate maufacturing", sense=pulp.LpMaximize)

In [4]:
# Set parameters
## Creates a list of the chocolates and ingredients
chocolates = ['A', 'B']
ingredients = ['Milk', 'Choco']

## Dictionaries of the costs, prices, milk and choco protion, maximal Ingredients are created
costs = {'Milk': 0.1, 
         'Choco': 0.3}

prices = {'A': 9,
          'B': 14}

portion = {'Milk': {'A': 1, 'B': 2},
           'Choco': {'A': 3,'B': 2}}

maxIngredients = {'Milk': 20, 
                  'Choco': 40}

In [5]:
# Create variables
chocolates_vars = LpVariable.dicts("chocolates_production", chocolates, lowBound=0, cat='Continuous')

In [6]:
# Set objective
prob += lpSum([chocolates_vars[chocolate] * prices[chocolate] for chocolate in chocolates]
            +[chocolates_vars[chocolate] * portion[ingredient][chocolate] * -costs[ingredient]
                 for chocolate in chocolates for ingredient in ingredients])

In [7]:
# Set constraints
for ingredient in ingredients:
    prob += lpSum(chocolates_vars[chocolate] * portion[ingredient][chocolate]
                  for chocolate in chocolates) <= maxIngredients[ingredient]

The quantifiers have become `` for...in loops`` and the summations have become calls to ``lpSum``

In [8]:
prob

chocolate maufacturing:
MAXIMIZE
8.0*chocolates_production_A + 13.200000000000001*chocolates_production_B + 0.0
SUBJECT TO
_C1: chocolates_production_A + 2 chocolates_production_B <= 20

_C2: 3 chocolates_production_A + 2 chocolates_production_B <= 40

VARIABLES
chocolates_production_A Continuous
chocolates_production_B Continuous

In [8]:
# Solve problem
prob.solve()
LpStatus[prob.status]

'Optimal'

In [9]:
for variable in prob.variables():
    print ("{} = {}".format(variable.name, variable.varValue))

chocolates_production_A = 10.0
chocolates_production_B = 5.0


In [10]:
print (pulp.value(prob.objective))

146.0
